In [28]:
#.env에서 API키 가져오기
import os
from dotenv import load_dotenv
load_dotenv('.env')
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')


In [ ]:
## 지금 작업중 (2/26 기준)

In [41]:
import requests
import time
import json
import os

# 네이버 API 환경변수 가져오기
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

# ✅ 검색할 소설 키워드 목록
query_list = ["한국 소설", "일본 소설", "영미 소설"]

# 📌 API 요청 함수
def search_books(query, start=1, display=100):
    url = "https://openapi.naver.com/v1/search/book.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,  # 최대 100개 가져오기
        "start": start,  # 페이징 처리
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()["items"]
    else:
        print(f"⚠️ API 요청 실패 ({response.status_code}): {query}, start={start}")
        return []

# 📌 전체 데이터 저장할 리스트
all_books = []
exclude_keywords = ["단편", "소설집", "단편선", "앤솔러지", "모음집", "옴니버스","자서전","시집","에세이","수필"]

# ✅ 여러 검색어로 데이터 수집
for query in query_list:
    for start in range(1, 1000, 100):  # 1000권 이상 확보 시도
        books = search_books(query, start)
        if not books:
            break  # 더 이상 결과 없음
        
        # 🔹 단편 소설 제외 (제목 또는 설명에 특정 키워드 포함 시 필터링)
        filtered_books = []
        for book in books:
            title = book["title"]
            description = book["description"]
            author = book.get("author", "알 수 없음")  # 🔹 작가 정보 가져오기
            
            # 📌 제외 키워드가 제목이나 설명에 포함되지 않은 경우만 저장
            if not any(keyword in title or keyword in description for keyword in exclude_keywords):
                filtered_books.append({
                    "title": title.split("(")[0].strip(),  # 🔹 제목에서 부가 정보 제거
                    "description": description.replace(".", ".\n"),  # 🔹 줄바꿈 추가
                    "author": author,  # 🔹 작가 정보 포함
                    "summary": ""  # 🔹 줄거리(요약)는 비워둠
                })

        all_books.extend(filtered_books)
        time.sleep(0.5)  # API 호출 제한 방지를 위해 대기

# ✅ 중복 제거 (책 제목 기준)
unique_books = {book["title"]: book for book in all_books}.values()

# 📌 JSON 파일로 저장
output_file = "filtered_novels.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(unique_books), f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(unique_books)}권의 장편 소설 데이터가 '{output_file}'에 저장되었습니다!")


✅ 총 1637권의 장편 소설 데이터가 'filtered_novels.json'에 저장되었습니다!


In [ ]:
import json
import random

# 📌 1. 전체 1637권 데이터 로드
with open("filtered_novels.json", "r", encoding="utf-8") as f:
    all_books = json.load(f)

# ✅ 2. 랜덤으로 200권 선택 (라벨링용)
random.shuffle(all_books)  # 랜덤 셔플
labeled_books = all_books[:200]  # 앞에서 200권 선택

# ✅ 3. 나머지 1437권 (모델 자동 요약용)
unlabeled_books = all_books[200:]

# 📌 4. 라벨링 데이터 저장
with open("labeling_novels.json", "w", encoding="utf-8") as f:
    json.dump(labeled_books, f, ensure_ascii=False, indent=4)

# 📌 5. 모델 요약용 데이터 저장
with open("unlabeled_books.json", "w", encoding="utf-8") as f:
    json.dump(unlabeled_books, f, ensure_ascii=False, indent=4)

print(f"✅ 랜덤 200권 라벨링 완료! ('labeling_novels.json' 저장)")
print(f"✅ 나머지 1437권 모델 요약 준비 완료! ('unlabeled_books.json' 저장)")


✅ 랜덤 200권 라벨링 완료! ('labeling_novels.json' 저장)
✅ 나머지 1481권 모델 요약 준비 완료! ('unlabeled_books.json' 저장)


In [ ]:
import json
import random

# 📌 1. 추가 라벨링 위해 1481권 데이터 로드
with open("unlabeled_books.json", "r", encoding="utf-8") as f:
    all_books = json.load(f)

# ✅ 2. 랜덤으로 100권 선택 (라벨링용)
random.shuffle(all_books)  # 랜덤 셔플
labeled_books = all_books[:100]  # 앞에서 200권 선택

# ✅ 3. 나머지 1381권 (모델 자동 요약용)
unlabeled_books = all_books[100:]

# 📌 4. 라벨링 데이터 저장
with open("labeling_novels_2.json", "w", encoding="utf-8") as f:
    json.dump(labeled_books, f, ensure_ascii=False, indent=4)

# 📌 5. 모델 요약용 데이터 저장
with open("unlabeled_books_final.json", "w", encoding="utf-8") as f:
    json.dump(unlabeled_books, f, ensure_ascii=False, indent=4)

print(f"✅ 랜덤 100권 라벨링 완료! ('labeling_novels_2.json' 저장)")
print(f"✅ 나머지 1337권 모델 요약 준비 완료! ('unlabeled_books_final.json' 저장)")
# labeling_novels_2.json + labeling_novels.json 합치고 unlabeled_books_2.json에서는 summary 제외해야함

✅ 랜덤 100권 라벨링 완료! ('labeling_novels_2.json' 저장)
✅ 나머지 1381권 모델 요약 준비 완료! ('unlabeled_books_final.json' 저장)


In [5]:
#summary 빼고 저장
import json

# ✅ 1️⃣ 기존 JSON 파일 로드
input_file = "unlabeled_books_final.json"
with open(input_file, "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 2️⃣ 모든 책에서 `summary` 필드 제거
for book in books_data:
    book.pop("summary", None)  # 🔹 `summary` 필드가 있으면 삭제

# ✅ 3️⃣ 새로운 JSON 파일로 저장
output_file = "unlabeled_books_no_summary.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(books_data, f, ensure_ascii=False, indent=4)

print(f"✅ `summary` 필드 제거 완료! 결과가 '{output_file}' 파일에 저장되었습니다.")


✅ `summary` 필드 제거 완료! 결과가 'unlabeled_books_no_summary.json' 파일에 저장되었습니다.


In [6]:
## 라벨링 데이터 중 "summary": "줄거리 없음"인 데이터 제거
import json

# ✅ 1️⃣ 기존 라벨링 데이터 로드
with open("labeling_novels.json", "r", encoding="utf-8") as f:
    labeled_books = json.load(f)

# ✅ 2️⃣ "줄거리 없음" 데이터 제거
filtered_books = [book for book in labeled_books if book["summary"].strip() != "줄거리 없음"]

# ✅ 3️⃣ 새로운 JSON 파일로 저장
output_file = "labeling_novels_filtered.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_books, f, ensure_ascii=False, indent=4)

print(f"✅ '줄거리 없음' 데이터를 제거하고, 결과를 '{output_file}' 파일에 저장했습니다!")


✅ '줄거리 없음' 데이터를 제거하고, 결과를 'labeling_novels_filtered.json' 파일에 저장했습니다!
